# Criação dos Exemplos

In [6]:
from mgd import pipelines
from mgd import filters
from itertools import chain
import pandas as pd

In [2]:
mapper = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz': 5,
    'metal': 6,
    'pop': 7,
    'reggae': 8,
    'rock': 9
}


In [3]:
examples = []
directories = list(mapper.keys())

for d in directories:
    examples.append(pipelines.extract_examples_from_dir(f'../gtzan/raw/{d}'))

examples = list(chain.from_iterable(examples))

In [12]:
ss_fields = filters.SummaryStatistics._fields

data = {
    'song_id': [],
    'label': []
}

sc_dict = {f'spectral_centroid_{s}': [] for s in ss_fields}
sf_dict = {f'spectral_flatness_{s}': [] for s in ss_fields}
sr_dict = {f'spectral_roll_of_{s}': [] for s in ss_fields}
mfcc_dict = {f'mfcc_{i}_{s}': [] for i in range(1, 13) for s in ss_fields}
tonnetz_dict = {f'tonnetz_{i}_{s}': [] for i in range(1, 7) for s in ss_fields}

data = data | sc_dict | sf_dict | sr_dict | mfcc_dict | tonnetz_dict


In [13]:
for e in examples:
    data['song_id'].append(e.song_id)
    data['label'].append(mapper[e.label])

    for ss in ss_fields:
        data[f'spectral_centroid_{ss}'].append(
            getattr(e.features.sc, ss).item(0))
        data[f'spectral_flatness_{ss}'].append(
            getattr(e.features.sf, ss).item(0))
        data[f'spectral_roll_of_{ss}'].append(
            getattr(e.features.sr, ss).item(0))

        for i in range(1, 13):
            mffc_str = f'mfcc_{i}_{ss}'
            data[mffc_str].append(getattr(e.features.mfcc, ss).item(i))

        for i in range(1, 7):
            tonnetz_str = f'tonnetz_{i}_{ss}'
            data[tonnetz_str].append(getattr(e.features.tonnetz, ss).item(i-1))


In [15]:
examples_df = pd.DataFrame(data)
examples_df.to_csv(f'../gtzan/processed/mgd_raw.csv', index=False)